In [1]:
from glob import glob
from functions import create_sentiment_dataset

In [2]:
files = glob('data/Multi Domain Sentiment/processed_acl/*/*')

In [3]:
df = create_sentiment_dataset(files)
df

,raw_text,label,text,folder,file
0,gaps:1 well:1 it_together:1 a_stack:1 the_cd:1...,negative,gaps well it together a stack the cd bott...,electronics,negative.review
1,save_your:2 steady_on:1 save:2 picture:1 your_...,negative,save your steady on save picture your mone...,electronics,negative.review
2,i:2 slightest_smudge:1 nice_for:1 errors:1 pla...,negative,i slightest smudge nice for errors player ...,electronics,negative.review
3,but_i:1 two:1 i:2 even:1 without:1 one:1 inexp...,negative,but i two i even without one inexpensive...,electronics,negative.review
4,failure:1 people_should:1 my_software:1 and_sa...,negative,failure people should my software and save ...,electronics,negative.review
...,...,...,...,...,...
27672,mass:1 specifically_references:1 science:1 rel...,negative,mass specifically references science releva...,books,unlabeled.review
27673,reviewer's_comments:1 to_pick:1 comments:1 and...,negative,reviewer's comments to pick comments and ul...,books,unlabeled.review
27674,x-ers:1 entry-level:1 can_dip:1 from_beginning...,positive,x-ers entry-level can dip from beginning v...,books,unlabeled.review
27675,your:1 well:1 around_for:1 you're_coming:1 for...,positive,your well around for you're coming for mor...,books,unlabeled.review
